In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import sklearn as sci
%matplotlib inline
%load_ext autoreload
%autoreload

df_ori = pd.read_csv('adyen-dataset.csv')
from lib.sampling import subsampling

# Subsample non_fraudulent transactions records so we have balanced dataset
df_fraudulent = df_ori[df_ori['has_fraudulent_dispute'] == True]
df_non_fraudulent = df_ori[df_ori['has_fraudulent_dispute'] == False]
subsample_index= subsampling(df_non_fraudulent.index, len(df_fraudulent))
df_non_fraudulent_subsample  = df_non_fraudulent.loc[subsample_index,:]
df_sample = pd.concat([df_non_fraudulent_subsample,df_fraudulent], axis=0)
df_sample.shape

(21432, 20)

In [2]:
df_sample.head()
df_sample = df_sample[:2000]

In [3]:
df = df_sample[['ip_address','email_address','card_number']]

In [4]:
def nb_common_attr(transaction1, transaction2):
    assert transaction1.size == transaction2.size
    merged = [*transaction1.values, *transaction2.values]
    nb_common_attr = transaction1.size*2 - len(set(merged))
    return nb_common_attr

In [5]:
import itertools

data = []
for i,j in itertools.combinations(df.index, 2):
    data.append([i, j, nb_common_attr(df.loc[i],df.loc[j])])
